## Riot API 발급
https://developer.riotgames.com

In [23]:
import pandas as pd
from tqdm import tqdm
import time

#API 요청
import requests

import plotly.express as px
import plotly.graph_objects as go

## data 불러오기

In [24]:
# riot_api 토큰 입력(매일 갱신해야한다.)
riot_token = 'RGAPI-cab1dbec-984a-436e-8585-2b791de50eb6'
nickname = input('nickname을 입력하세요 : ')

nickname을 입력하세요 : 친일파김구


In [25]:
# 소환사 정보
req_url_summoner = f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{nickname}?api_key={riot_token}'

summoner_data = requests.get(req_url_summoner)
# summoner_data.json()

In [26]:
summoner_id = summoner_data.json()['id'] # 지역별 소환사 고유 아이디
puuid = summoner_data.json()['puuid'] #  소환사 고유 아이디

## 게임 기록 가져오기

In [27]:
start = 0
count = 100
req_url_match = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}&api_key={riot_token}'
match_data = requests.get(req_url_match)

In [28]:
def get_my_index(get_data,puuid):
    if 'metadata' not in get_data.json().keys():
        print(get_data.json())
    else:
        for idx,p_id in enumerate(get_data.json()['metadata']['participants']):
            if p_id==puuid:
                my_idx = idx
                return my_idx
            else:
                continue
                
print(f'전체 불러온 게임 수 : {len(match_data.json())}')

전체 불러온 게임 수 : 100


In [29]:
# 클래식 게임만 담을 리스트
classic_game_list = []
# 데이터를 담을 빈 데이터 프레임 생성
total_df = pd.DataFrame()
# api 요청 횟수 조절을 위한 상수 1초에 20회 2분에 100회
cnt = 0

# 불러온 매치 아이디로 게임 데이터 요청해서 dataframe에 데이터 채우기
for match in tqdm(match_data.json()):
    
    # match_data 리스트에서 match 하나를 불러와서 데이터 요청
    match_id = match
    req_url_match_info = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={riot_token}'
    match_info = requests.get(req_url_match_info) # 매치 데이터
    
    try:
        if 'metadata' in match_info.json().keys():
            if match_info.json()['info']['gameMode']=='CLASSIC' and match_info.json()['info']['gameType']=='MATCHED_GAME':
                    classic_game_list.append(match_id)
                    # match에서 나의 번호 찾기
                    my_idx = get_my_index(get_data=match_info,puuid=puuid)

                    # 유저들 정보 temp_df에 할당
                    temp_df = pd.DataFrame.from_dict(match_info.json()['info']['participants'])

                    # dataframe에서 my_idx만 추출
                    my_series = temp_df.iloc[my_idx]
                    my_df = pd.DataFrame(my_series).T

                    # 게임길이 total_df에 추가
                    game_length_dict = {}
                    game_length_dict['match_id'] = match_id
                    game_length_dict['time'] = my_df['challenges'][my_idx]['gameLength']
                    game_length_dict['type'] = '총 게임 시간'
                    total_df = total_df.append(game_length_dict,ignore_index=True)

                    # 죽은 시간 total_df에 추가
                    total_time_spend_dead_dict = {}
                    total_time_spend_dead_dict['match_id'] = match_id
                    total_time_spend_dead_dict['time'] = my_df['totalTimeSpentDead'][my_idx]
                    total_time_spend_dead_dict['type'] = '죽어있던 시간'
                    total_df = total_df.append(total_time_spend_dead_dict,ignore_index=True)
                    
    except:
        print(match_info.json())
        
    cnt += 1
    
    if cnt==19:
        time.sleep(1)
        cnt = 0

100%|█████████████████████████████████████████| 100/100 [00:41<00:00,  2.41it/s]


In [30]:
print(f'일반, 랭크 게임 수 : {len(classic_game_list)}')

일반, 랭크 게임 수 : 99


In [31]:
total_df.pivot(index='match_id',columns='type',values='time')

type,죽어있던 시간,총 게임 시간
match_id,,
KR_5885032321,35.0,1165.244391
KR_5885067354,204.0,1367.125178
KR_5885194272,103.0,1183.139944
KR_5885303163,174.0,1735.180966
KR_5886343477,377.0,1846.714391
...,...,...
KR_5947604622,0.0,1434.289963
KR_5947676632,78.0,1311.747876
KR_5948035732,85.0,1251.522783


In [32]:
# 초 단위를 분 단위로 변경
total_df['time'] = total_df['time'].apply(lambda x : x/60)

In [33]:
total_df = total_df.drop(total_df.query('time<=4 and type=="총 게임 시간"').index[0])
fig = px.box(total_df, x="type", y="time", 
             labels={
                     "type": "시간의 종류",
                     "time": "시간 (분)",
                 },
             title="분포")
fig.show()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [34]:
# 시간 순으로 오름차순 정렬
test_df = total_df.sort_values(by=['type','time'],ascending = [False, True]).reset_index(drop=True)

In [35]:
# 데이터 프레임 형식 변경
test_df_pivot = test_df.pivot(index='match_id', columns='type', values='time')
test_df_pivot['rate'] = test_df_pivot['죽어있던 시간'] / test_df_pivot['총 게임 시간']
game_length_mean = test_df_pivot['총 게임 시간'].mean()
total_time_spend_dead_mean = test_df_pivot['죽어있던 시간'].mean()
dead_rate_mean = test_df_pivot.rate.mean()

In [36]:
fig = px.bar(test_df, x='match_id', y='time', color='type', barmode='group')

fig.add_trace(go.Scatter(
    x=[test_df['match_id'][20], test_df['match_id'][25]], 
    y=[40, 50],
    text=[f'평균 게임 길이 : {round(game_length_mean,2)}분',f'평균적으로 게임 시간의 {round(dead_rate_mean,2)}분을 죽어있습니다'],
    mode='text'))

fig.add_shape(type='line', x0=test_df['match_id'][0], x1=test_df['match_id'][len(test_df)-1], 
              y0=game_length_mean, y1=game_length_mean,
              line=dict(
                 color="red",
                 width=4,
                 dash="dashdot"))

fig.update_layout(
    title="총 게임 길이별 죽어 있던 시간 시각화",
    xaxis_title="매치 아이디",
    yaxis_title="시간 (분)",
    legend_title="시간 종류",
)



fig.show()